### Setup

In [1]:
chipyard_dir = "/tools/scratch/nayiri/power/chipyard-intech16-sep23"
exec(open(f"util.py").read())

In [2]:
my_rtl = 'RocketConfig'
# my_workloads = riscv_benchmarks
my_w = 'towers' # stick to one workload for now
runparams = {
    'rtl': my_rtl,
    'bsub': 'bsub -q rhel7',
    'overwrite': True,
}

### Run RTL Sim

In [3]:
print(f"""
cd {chipyard_dir}
source env.sh
cd vlsi
make sim-rtl-debug CONFIG={my_rtl} BINARY={workloads[my_w]['binary_path']}
""")


cd /tools/scratch/nayiri/power/chipyard-intech16-sep23
source env.sh
cd vlsi
make sim-rtl-debug CONFIG=RCP BINARY=/tools/scratch/nayiri/power/chipyard-intech16-sep23/.conda-env/riscv-tools/riscv64-unknown-elf/share/riscv-tests/benchmarks/towers.riscv



### Parse RTL Sim Waveforms

#### Header

In [4]:
# use any of the workloads to print header info + dump idcodes, since that info is the same across all workloads
w = 'towers'
waveform_fpath = get_waveform_path(w,my_rtl)
header_fpath = get_outfile_path('headers',rtl=my_rtl)
print(f"./print_header {waveform_fpath} > {header_fpath}")


./print_header /tools/scratch/nayiri/power/chipyard-intech16-sep23/vlsi/output/chipyard.harness.TestHarness.RCP/towers.fsdb > /bwrcq/C/nayiri/fsdb-parse/out/fsdb/headers/RCP.txt


#### Idcodes

In [5]:
dump_idcodes(rtl=my_rtl)

total signals: 21715
[num_signals] + [signal fsdb idcodes] --> /bwrcq/C/nayiri/fsdb-parse/out/fsdb/idcodes/RCP.txt
[num_signals] + [(signal fsdb idcode, signal name(s)), ...] --> /bwrcq/C/nayiri/fsdb-parse/out/fsdb/idcodes/signames-RCP.txt


#### Dump signal value for each workload

In [6]:
# first, get idcode of instruction signal in Rocket core
#   from the Verilog we know this signal name is the path below (this isn't elegant but it works!)
signal_name = "TestDriver/testHarness/chiptop0/system/tile_prci_domain/tile_reset_domain_tile/core/_csr_io_trace_0_insn[31:0]"
signames_idcodes_fpath = get_outfile_path('idcodes','signames',rtl=my_rtl)
with open(signames_idcodes_fpath,'r') as f:
    for line in f.readlines():
        words = line.split()
        if len(words) < 2 or not words[0].isnumeric(): continue
        if signal_name in words[1]: idcode = int(words[0]); break
waveform_fpath = get_waveform_path(my_w,my_rtl)
bin_file = get_outfile_path('sigvals',my_w,rtl=my_rtl, idcode=idcode)
print(f"""./dump_signal --fsdb {waveform_fpath} --idcode {idcode}  --out {bin_file}""")

./dump_signal --fsdb /tools/scratch/nayiri/power/chipyard-intech16-sep23/vlsi/output/chipyard.harness.TestHarness.RCP/towers.fsdb --idcode 175523  --out /bwrcq/C/nayiri/fsdb-parse/out/fsdb/RCP/towers-idcode_175523.bin


### Load signal value

In [7]:
times, sigvals = get_workload_sigvals(my_w,rtl=my_rtl,idcode=idcode)

In [8]:
# visually compare this output with spike log dump from RTL sim
for s in sigvals[:10]:
    print(bin(s))

0b10110111111001011000010100100011
0b11101110
0b10100010111
0b11111100000001010000010100010011
0b110000010101010001000001110011
0b110000000100000010001011110011
0b1000001001000101101001010010011
0b100101111001010010011
0b101000010001100011
0b110000001100000001000001110011
